# Fingerprint segmentation module

In [1]:
import numpy as np
import cv2 as cv
import tensorflow as tf
from utils import show
from ipywidgets import interact
import time
from IPython.display import clear_output
import json

# TODO ???
import os, sys
current = os.path.dirname(os.path.realpath("."))
sys.path.append(current)

import pyfing as pf

2023-10-09 11:39:20.907331: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-09 11:39:20.907415: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-09 11:39:20.907451: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 11:39:20.917910: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Paths
PATH_FVC = '../../datasets/'
PATH_PARAMS = '../parameters/segmentation/'
PATH_GT = '../../datasets/segmentationbenchmark/groundtruth/'
PATH_TESTS = '../../results/'

# Error analysis

For each database, fingerprints are sorted according to the error rate.

In [27]:
@interact(year=[2000,2002,2004], db=[1,2,3,4])
def choose_db(year, db):
    subset = 'a'
    with open(f'{PATH_TESTS}fvc{year}_db{db}_{subset}_res.txt', 'r') as filehandle:
        results = json.load(filehandle)

    results = sorted(results, key=lambda r: r[2][0])
    seg.parameters = segmentation.GradMagSegmentationParameters.load(f'{PATH_PARAMS}fvc{year}_db{db}_b_best_params.txt')
    
    @interact(i=(0, len(results)-1))
    def show_result(i=0):
        i, j, (err, dc) = results[i]
        img = cv.imread(f'{PATH_FVC}fvc{year}/db{db}_{subset}/{i}_{j}.png', cv.IMREAD_GRAYSCALE)
        gt = cv.bitwise_not(cv.imread(f'{PATH_GT}fvc{year}_db{db}_im_{i}_{j}seg.png', cv.IMREAD_GRAYSCALE))
        mask = seg.run(img)
        v_mask = cv.merge((gt, mask, cv.bitwise_not(img)))
        show((img,f'{i}_{j}'), (mask, 'Result'), (gt, 'Ground truth'), (v_mask, f'Err = {err:.2f}% DC = {dc:.3f}'))

interactive(children=(Dropdown(description='year', options=(2000, 2002, 2004), value=2000), Dropdown(descripti…

## 3. Parameter optimization

Optimizazion of the parameters on each FVC database set "B":

In [ ]:
def optimize_parameters_in_ranges(year, db, subset, images, gt, best_parameters, ranges):    
    min_err = 100 if best_parameters is None else average_err_on_db(images, gt, best_parameters)
    for w in ranges[0]:
        for p in ranges[1]:  
            for t in ranges[2]:
                for cc in ranges[3]:
                    for oc in ranges[4]:
                        parameters = segmentation.Parameters(w, p, t, cc, oc)
                        e = average_err_on_db(images, gt, parameters)                
                        if e < min_err:
                            min_err = e
                            best_parameters = parameters
                            # saves best parameters to file
                            best_parameters.save(f'../test/fvc{year}_db{db}_{subset}_params.txt')
                        print(f'{year}-{db}-{subset}: {len(images)} fingerprints')
                        print(f'{parameters}\n-> {e:.2f}%\n(best: {best_parameters}\n-> {min_err:.2f}%)')
                        clear_output(True)
    return best_parameters

def optimize_parameters(year, db, subset):
    images = load_db(year, db, subset)
    gt = load_gt(year, db, subset)
    p = None
    try:
        p = segmentation.load_parameters(f'{PATH_TESTS}fvc{year}_db{db}_b_params.txt')
        print(f'Loaded best parameters: {p}')
    except:
        print('Best parameters not found')
    p = optimize_parameters_in_ranges(year, db, subset, images, gt, p,
                                      [
                                          range(3,19,2),
                                          [85,90,95],
                                          [0.15,0.20,0.25,0.30],
                                          [7],
                                          [13],
                                      ])
    p = optimize_parameters_in_ranges(year, db, subset, images, gt, p,
                                      [
                                          [p.window_size],
                                          range(75,100,5),
                                          np.arange(0.05,0.4,0.01),
                                          [7],
                                          [13],
                                      ])
    p = optimize_parameters_in_ranges(year, db, subset, images, gt, p,
                                      [
                                          [p.window_size],
                                          [p.percentile],
                                          [p.threshold],
                                          range(0,15,3),
                                          range(0,24,3),
                                      ])

In [ ]:
for y, db, subset in TRAIN_DATASETS:
    optimize_parameters(y, db, subset)